# Recommender ChatBot: Evaluation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

BASE_PATH    = '../..'
API_PATH     = f'{BASE_PATH}/chat-bot-api'
LIB_PATH     = f'{BASE_PATH}/lib'
DATASET_PATH = f'{BASE_PATH}/datasets'

os.environ['TMP_PATH']         = f'{BASE_PATH}/tmp'
os.environ['DATASET_PATH']     = f'{BASE_PATH}/datasets'
os.environ['WEIGHTS_PATH']     = f'{BASE_PATH}/weights'
os.environ['METRICS_PATH']     = f'{BASE_PATH}/metrics'

In [3]:
import sys
sys.path.append(LIB_PATH)
sys.path.append(API_PATH)

import pytorch_common.util as pu

import client

import util as ut
from faker import Faker

# Common Functions and Classes

In [4]:
import pandas as pd
import numpy as np

def genres_count(interactions_test_set, items, user_id):
    df = interactions_test_set[interactions_test_set['user_id'] == user_id]

    item_ids = df['item_id'].unique()
    
    genres = []
    for item_id in item_ids:
        genres.extend(items[items['movie_id'] == item_id]['movie_genres'].tolist()[0])

    return pd.Series(genres).value_counts().reset_index(name='count')



def releases(interactions_test_set, items, user_id):
    df = interactions_test_set[interactions_test_set['user_id'] == user_id]

    return np.sort(items[items['movie_id'].isin(df['item_id'].unique())]['movie_release_year'].unique())

In [16]:
def create_fake_profiles(interactions_test_set, items):
    fake = Faker()

    to_email = lambda name: name.lower().replace(' ', '.') + '@gmail.com'

    user_ids = interactions_test_set['user_id'].unique()
    
    profiles = []
    
    for user_id in user_ids:
        name  = fake.name()
        email  = to_email(name)
        genres_count_df = genres_count(interactions_test_set, items, user_id=user_id)
        genres = genres_count_df[genres_count_df['count'] >= 10]['index'].tolist()
    
        release = str(releases(interactions_test_set, items, user_id=user_id)[0])
        
        profile = client.UserProfileDto(
            name             = name,
            email            = to_email(name),
            preferred_from   = release,
            preferred_genres = genres
        )
    
        profiles.append(profile)

    return profiles

## Setup

In [5]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

**Step 1**: Load interactions test set.

In [6]:
interactions_test_set = ut.load_df(f'{DATASET_PATH}/interactions_test_set.json')

In [7]:
items = ut.load_df(f'{DATASET_PATH}/pre-processed-movies.json')

**Step 2**: Create **Recommendation Chatbot API** client.

In [8]:
api_client = client.RecChatBotV1ApiClient()

In [14]:
api_client.health

2024-03-18 21:23:16,852 - INFO - GET http://nonosoft.ddns.net:8080/api/v1/health



{
    'chatbot_api': True,
    'ollama_api': True,
    'airflow': {'metadatabase': True, 'scheduler': True},
    'mongo_database': True,
    'choma_database': True
}

**Step 3**: Create profiles for test users.

In [17]:
profiles = create_fake_profiles(interactions_test_set, items)

In [19]:
[api_client.remove_interactions_by_user_id(profile.email) for profile in profiles]
[api_client.add_profile(profile) for profile in profiles]
None

2024-03-18 21:26:42,388 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/rebecca.douglas@gmail.com
2024-03-18 21:26:42,411 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/rebecca.gordon@gmail.com
2024-03-18 21:26:42,433 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/blake.kim@gmail.com
2024-03-18 21:26:42,457 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/darlene.holder@gmail.com
2024-03-18 21:26:42,481 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/mallory.nichols@gmail.com
2024-03-18 21:26:42,503 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/corey.weaver@gmail.com
2024-03-18 21:26:42,524 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/joann.henry@gmail.com
2024-03-18 21:26:42,546 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/kerry.white@gmail.com
2024-03-18 21:26:42,570 - INFO - DELETE htt

In [20]:
chatbot_settings = {
    'llm'                                   : 'llama2-7b-chat',
    'retry'                                 : 2,
    'plain'                                 : False,
    'include_metadata'                      : True,
    'rag': {
        'shuffle'                           : True,
        'candidates_limit'                  : 50,
        'llm_response_limit'                : 50,
        'recommendations_limit'             : 5,
        'similar_items_augmentation_limit'  : 5,
        'not_seen'                          : True
    },
    'collaborative_filtering': {
        'shuffle'                           : False,
        'candidates_limit'                  : 50,
        'llm_response_limit'                : 50,
        'recommendations_limit'             : 5,
        'similar_items_augmentation_limit'  : 5,
        'text_query_limit'                  : 5000,
        'k_sim_users'                       : 10,
        'random_selection_items_by_user'    : 0.5,
        'max_items_by_user'                 : 10,
        'min_rating_by_user'                : 3.5,
        'not_seen'                          : True
    }
}

In [21]:
%%time

query = {
    'message': {
        'author': profiles[0].email,
        'content': 'I want see marvel movies'
    },
    'settings': chatbot_settings
}

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.29 µs


In [37]:
result = api_client.recommendations(query)
result.items

2024-03-18 21:44:48,318 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/recommendations



[
    {
    "description": "A superhero team must come together to save the world from an alien invasion.",
    "genres": [
        "action",
        "adventure",
        "sci-fi",
        "imax"
    ],
    "metadata": {
        "db_item": {
            "id": "89745",
            "query_sim": 0.13897466659545898,
            "rating": 3.612244898,
            "release": "2012",
            "release_sim": 1.0,
            "title": "Avengers, The",
            "title_sim": 0.9999999999999998
        },
        "result_item": {
            "position": 1,
            "title": "The avengers"
        }
    },
    "poster": "http://image.tmdb.org/t/p/w500/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg",
    "release": "2012",
    "title": "Avengers, The",
    "votes": [
        "http://nonosoft.ddns.net:8080/api/v1/interactions/make/rebecca.douglas@gmail.com/89745/1",
        "http://nonosoft.ddns.net:8080/api/v1/interactions/make/rebecca.douglas@gmail.com/89745/2",
        "http://nonosoft.ddns.net:8080/ap

In [38]:
result.items[0].vote(5)
api_client.interactions_by_user(profiles[0].email)

2024-03-18 21:45:17,729 - INFO - GET http://nonosoft.ddns.net:8080/api/v1/interactions/make/rebecca.douglas@gmail.com/89745/5
2024-03-18 21:45:17,783 - INFO - 204



[
    {
        'user_id': 'rebecca.douglas@gmail.com',
        'item_id': '102125',
        'rating': 5.0,
        'timestamp': '2024-03-18 21:41:13'
    },
    {
        'user_id': 'rebecca.douglas@gmail.com',
        'item_id': '2153',
        'rating': 5.0,
        'timestamp': '2024-03-18 21:44:10'
    },
    {
        'user_id': 'rebecca.douglas@gmail.com',
        'item_id': '89745',
        'rating': 5.0,
        'timestamp': '2024-03-18 21:44:50'
    }
]

In [39]:
result = api_client.recommendations(query)
result.items

2024-03-18 21:45:17,818 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/recommendations



[
    {
    "description": "Tony stark must confront his demons and defeat a powerful enemy in this action-packed sequel.",
    "genres": [
        "action",
        "adventure",
        "sci-fi"
    ],
    "metadata": {
        "db_item": {
            "id": "59315",
            "query_sim": 0.19898885488510132,
            "rating": 3.8469387755,
            "release": "2008",
            "release_sim": 0.0,
            "title": "Iron Man",
            "title_sim": 0.9999999999999998
        },
        "result_item": {
            "position": 2,
            "title": "Iron man 3"
        }
    },
    "poster": "http://image.tmdb.org/t/p/w500/78lPtwv72eTNqFW9COBYI0dWDJa.jpg",
    "release": "2013",
    "title": "Iron Man",
    "votes": [
        "http://nonosoft.ddns.net:8080/api/v1/interactions/make/rebecca.douglas@gmail.com/59315/1",
        "http://nonosoft.ddns.net:8080/api/v1/interactions/make/rebecca.douglas@gmail.com/59315/2",
        "http://nonosoft.ddns.net:8080/api/v1/inter